In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
import imutils
import cv2, sys, os
from tensorflow.keras.models import load_model
import numpy as np
import dlib
import matplotlib.pyplot as plt
from imutils import face_utils

### 사진에서 얼굴 인식

In [3]:
# 가중치 파일 경로
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

if faceCascade.empty():
    print('XML load failed')
    sys.exit()
    
img = cv2.imread('./img/flower.png')
    
if img is None:
    print('image read failed')
    sys.exit()

cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 얼굴에 박스를 만드는 함수(얼굴을 검출)
faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.1,
                                    minNeighbors = 3, 
                                    minSize = (30, 30))

# 각 행마다 (x, y, w, h)를 받아서 사각형을 그리는 코드
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)

cv2.imshow('img', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
img = cv2.imread('./img/animal-7089224_960_720.webp')

model = '../facedetection/opencv_face_detector_uint8.pb'
config = '../facedetection/opencv_face_detector.pbtxt'
emotion_model_path = './_mini_XCEPTION.102-0.66.hdf5'

face_detect_net = cv2.dnn.readNet(model, config)

emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]

cv2.namedWindow('image', cv2.WINDOW_NORMAL)

if face_detect_net.empty():
    print('net load failed')
    sys.exit()
    
# img = cv2.resize(img, (300, 300))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB = False)
face_detect_net.setInput(blob)
out = face_detect_net.forward()

detect = out[0, 0, :, :]

h, w = img.shape[:2]

canvas = np.zeros((250, 300, 3), dtype="uint8")

ImageClone = img.copy()

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        x1 = int(detect[i, 3]*w)
        y1 = int(detect[i, 4]*h)
        x2 = int(detect[i, 5]*w)
        y2 = int(detect[i, 6]*h)
        
        cv2.rectangle(ImageClone, (x1, y1), (x2, y2), (0, 0, 255), 2)
        
        roi = gray[y1:y1 + y2, x1:x1 + x2]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
    else: continue
            
for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
    text = "{}: {:.2f}%".format(emotion, prob * 100)
    
    w = int(prob * 300)
    cv2.rectangle(canvas, (7, (i * 35) + 5),
                  (w, (i * 35) + 35), (0, 0, 255), -1)        
    cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)    
    cv2.putText(ImageClone, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.rectangle(ImageClone, (x1, y1), (x2, y2),
                  (0, 0, 255), 2)
    
cv2.imshow('image', ImageClone)
cv2.imshow("Probabilities", canvas)

cv2.waitKey()
cv2.destroyAllWindows()

![image](maxresdefault.jpg)

### 캠으로 얼굴 인식

In [5]:
# parameters for loading data and images
detection_model_path = './haarcascade_frontalface_default.xml'
emotion_model_path = './_mini_XCEPTION.102-0.66.hdf5'

# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]


#feelings_faces = []
#for index, emotion in enumerate(EMOTIONS):
   # feelings_faces.append(cv2.imread('emojis/' + emotion + '.png', -1))

# starting video streaming
cv2.namedWindow('face')
camera = cv2.VideoCapture(0)
while True:
    frame = camera.read()[1]
    #reading the frame
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,
                                            minNeighbors=5,minSize=(30,30),
                                            flags=cv2.CASCADE_SCALE_IMAGE)
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
                    # Extract the ROI of the face from the grayscale image, resize it to a fixed 28x28 pixels, and then prepare
            # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
    else: continue

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # draw the label + probability bar on the canvas
               # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5),
                (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2)
#    for c in range(0, 3):
#        frame[200:320, 10:130, c] = emoji_face[:, :, c] * \
#        (emoji_face[:, :, 3] / 255.0) + frame[200:320,
#        10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)


    cv2.imshow('face', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

haarcascade_frontalface_default.xml의 영상처리 속도가 느려 res10_300x300_ssd_iter_140000_fp16.caffemodel 모델로 대체

In [3]:
# parameters for loading data and images

model = '../facedetection/res10_300x300_ssd_iter_140000_fp16.caffemodel'
config = '../facedetection/deploy.prototxt'
emotion_model_path = './_mini_XCEPTION.102-0.66.hdf5'

# hyper-parameters for bounding boxes shape
# loading models

emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]

# print(type(emotion_classifier))
# print(emotion_classifier)

# feelings_faces = []
# for index, emotion in enumerate(EMOTIONS):
#    feelings_faces.append(cv2.imread('emojis/' + emotion + '.png', -1))

# starting video streaming
cv2.namedWindow('face')
camera = cv2.VideoCapture(0)

if not camera.isOpened():
    print('video open failed')
    sys.exit()
    
net = cv2.dnn.readNet(model, config)
    
while True:
    ret, frame = camera.read()
    
    if not ret:
        print('net load failed')
        break
        
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    #reading the frame
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123))
    net.setInput(blob)
    out = net.forward()
    
    detect = out[0, 0, :, :]
    
    h, w = frame.shape[:2]
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    
    frameClone = frame.copy()
    
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        
        if confidence > 0.5:
            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
            roi = gray[y1:y1 + y2, x1:x1 + x2]
#             print('roi_1 : ', roi.shape)
#             print('roi_1 : ', roi.dtype)
            roi = cv2.resize(roi, (64, 64))
#             print('roi_2 : ', roi.shape)
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
#             print(roi.shape)
            roi = np.expand_dims(roi, axis=0)
#             print('roi_3 : ', roi.shape); print()
            
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]
            
        else: continue

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # draw the label + probability bar on the canvas
               # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5),
                    (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                    (255, 255, 255), 2)
                cv2.putText(frameClone, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (x1, y1), (x2, y2),
                              (0, 0, 255), 2)
#    for c in range(0, 3):
#        frame[200:320, 10:130, c] = emoji_face[:, :, c] * \
#        (emoji_face[:, :, 3] / 255.0) + frame[200:320,
#        10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)


    cv2.imshow('face', frameClone)
    cv2.imshow("Probabilities", canvas)
    
    key = cv2.waitKey(30)
    if key == 27 or key == ord('q'):
        break

# print(frame.shape)
# print(gray.shape)
# print(roi.shape)
camera.release()
cv2.destroyAllWindows()

In [6]:
detector = dlib.cnn_face_detection_model_v1('dogHeadDetector.dat')
predictor = dlib.shape_predictor('landmarkDetector.dat')

img_path = './img/dog-7178060_960_720.jpg'
filename, ext = os.path.splitext(os.path.basename(img_path))
emotion_model_path = './_mini_XCEPTION.102-0.66.hdf5'

emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]

cv2.namedWindow('image', cv2.WINDOW_NORMAL)

img = cv2.imread(img_path)
# img = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)

img = cv2.resize(img, (300, 300))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

canvas = np.zeros((250, 300, 3), dtype="uint8")

dets = detector(img, upsample_num_times=1)
ImageClone = img.copy()

for i, d in enumerate(dets):

    x1, y1 = d.rect.left(), d.rect.top()
    x2, y2 = d.rect.right(), d.rect.bottom()

    cv2.rectangle(ImageClone, pt1=(x1, y1), pt2=(x2, y2),
                  thickness=2, color=(0,0,255), lineType=cv2.LINE_AA)
    
    roi = gray[y1:y1 + y2, x1:x1 + x2]
    roi = cv2.resize(roi, (64, 64))
    roi = roi.astype('float') / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)
    
    preds = emotion_classifier.predict(roi)[0]
    emotion_probability = np.max(preds)
    label = EMOTIONS[preds.argmax()]
    
    
for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
    text = "{}: {:.2f}%".format(emotion, prob * 100)
    
    w = int(prob * 300)
    cv2.rectangle(canvas, (7, (i * 35) + 5),
                  (w, (i * 35) + 35), (0, 0, 255), -1)        
    cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)    
    cv2.putText(ImageClone, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.rectangle(ImageClone, (x1, y1), (x2, y2),
                  (0, 0, 255), 2)
    
cv2.imshow('image', ImageClone)
cv2.imshow("Probabilities", canvas)

cv2.waitKey()
cv2.destroyAllWindows()